In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_train["Title"] = df_train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df_test["Title"] = df_test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df_train["Title"] = df_train["Title"].replace('Master', 'Master')
df_train["Title"] = df_train["Title"].replace('Mlle', 'Miss')
df_train["Title"] = df_train["Title"].replace(['Mme', 'Dona', 'Ms'], 'Mrs')
df_train["Title"] = df_train["Title"].replace(['Don','Jonkheer'],'Mr')
df_train["Title"] = df_train["Title"].replace(['Capt','Rev','Major', 'Col','Dr'], 'Millitary')
df_train["Title"] = df_train["Title"].replace(['Lady', 'Countess','Sir'], 'Honor')
df_test["Title"] = df_test["Title"].replace('Master', 'Master')
df_test["Title"] = df_test["Title"].replace('Mlle', 'Miss')
df_test["Title"] = df_test["Title"].replace(['Mme', 'Dona', 'Ms'], 'Mrs')
df_test["Title"] = df_test["Title"].replace(['Don','Jonkheer'],'Mr')
df_test["Title"] = df_test["Title"].replace(['Capt','Rev','Major', 'Col','Dr'], 'Millitary')
df_test["Title"] = df_test["Title"].replace(['Lady', 'Countess','Sir'], 'Honor')
titledummies = pd.get_dummies(df_train[['Title']], prefix_sep='_') #Title
df_train = pd.concat([df_train, titledummies], axis=1) 
ttitledummies = pd.get_dummies(df_test[['Title']], prefix_sep='_') #Title
df_test = pd.concat([df_test, ttitledummies], axis=1) 
df_train["Embarked"] = df_train['Embarked'].fillna('S')
df_test["Embarked"] = df_test['Embarked'].fillna('S')
dummies = pd.get_dummies(df_train[["Embarked"]], prefix_sep='_')
df_train = pd.concat([df_train, dummies], axis=1) 
dummies = pd.get_dummies(df_test[["Embarked"]], prefix_sep='_')
df_test = pd.concat([df_test, dummies], axis=1)
df_train["Fare"] = df_train["Fare"].fillna(np.median(df_train["Fare"]))
df_test["Fare"] = df_test["Fare"].fillna(np.median(df_test["Fare"]))
titles = ['Master', 'Miss', 'Mr', 'Mrs', 'Millitary','Honor']
for title in titles:
    age_to_impute = df_train.groupby('Title')['Age'].median()[title]
    df_train.loc[(df_train['Age'].isnull()) & (df_train['Title'] == title), 'Age'] = age_to_impute
    df_test.loc[(df_test['Age'].isnull()) & (df_test['Title'] == title), 'Age'] = age_to_impute
sex_mapping = {"male": 0, "female": 1}
df_train['Sex'] = df_train['Sex'].map(sex_mapping)
df_test['Sex'] = df_test['Sex'].map(sex_mapping)
df_train = df_train.drop(['Cabin'], axis = 1)
df_test = df_test.drop(['Cabin'], axis = 1)
df_train = df_train.drop(['Name'], axis = 1)
df_test = df_test.drop(['Name'], axis = 1)
cols = df_test.columns.tolist()
df_test = df_test[cols[:10] + ['Title_Honor'] + cols[10:-1]]
def encode_features(df_train, df_test):
    features = ['Title','Embarked','Ticket']    
    for feature in features:
        combined = pd.concat([df_train[feature], df_test[feature]], axis=0).astype(str)     
        le = LabelEncoder()
        le.fit(combined)     
        df_train[feature] = le.transform(df_train[feature].astype(str))
        df_test[feature]  = le.transform(df_test[feature].astype(str))  
    return df_train, df_test
df_train, df_test = encode_features(df_train.copy(), df_test.copy())
df_train.head()
df_test['Title_Honor'] = False
predictors = df_train.drop(['Survived', 'PassengerId'], axis=1)
target = df_train["Survived"]
X_train, X_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22)
model2 = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=5, min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    bootstrap=True,           
    class_weight='balanced',)
model2.fit(X_train,y_train)
prediction = model2.predict(df_test.drop(columns="PassengerId"))
test = pd.DataFrame( { 'PassengerId': df_test['PassengerId'] , 'Survived': prediction } )
test.shape
test.head()
test.to_csv( 'titanic_pred.csv' , index = False )

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/bl/1vx9mxbs4wb6r4dlwshm21240000gn/T/ipykernel_7101/49863852.py:1: SyntaxWarning: invalid escape sequence '\.'
  df_train["Title"] = df_train.Name.str.extract(' ([A-Za-z]+)\.', expand=False) #Creating new column name Title
/var/folders/bl/1vx9mxbs4wb6r4dlwshm21240000gn/T/ipykernel_7101/49863852.py:2: SyntaxWarning: invalid escape sequence '\.'
  df_test["Title"] = df_test.Name.str.extract(' ([A-Za-z]+)\.', expand=False) #Creating new column name Title
